# Rekonstruktion von Morse-Codes mithilfe von LSTM-Netwerken

Ziel dieses Projektes ist es, die Leerzeichen beziehungsweise die Pausen zwischen den Darstellungen einzelner Buchstaben im Morsecode mithilfe eines rekurrenten LSTM-Netzwerkes zu rekonstruieren. Obwohl durch das Weglassen der Leerzeichen im Prinzip Information verloren geht, ergeben meistens nur bestimmte Positionen der Leerzeichen sinnvolle Wörter.

In [1]:
import tensorflow as tf
import numpy as np

2023-03-12 14:48:13.588706: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Morse-Kodierung

Zuerst schreiben wir eine Funktion `to_morse`, die ein Wort in Morse-Code übersetzt

In [2]:
morse_code = {
    'A': '.-',     'B': '-...',   'C': '-.-.',   'D': '-..',    'E': '.',      'F': '..-.',
    'G': '--.',    'H': '....',   'I': '..',     'J': '.---',   'K': '-.-',    'L': '.-..',
    'M': '--',     'N': '-.',     'O': '---',    'P': '.--.',   'Q': '--.-',   'R': '.-.',
    'S': '...',    'T': '-',      'U': '..-',    'V': '...-',   'W': '.--',    'X': '-..-',
    'Y': '-.--',   'Z': '--..' }

In [3]:
def to_morse_word(word):
    word = word.upper()
    if not all([x in morse_code for x in word]):
        raise Exception("Wort enthält Zeichen außerhalb des englischen Alphabets")
    return ' '.join([morse_code[x] for x in word])
def to_morse(phrase):
    return 'X'.join([to_morse_word(word) for word in phrase.split(" ")])

Test der Funktion:

In [4]:
to_morse("SOS V")

'... --- ...X...-'

## Eingabe und Ausgabe des neuronalen Netzwerkes

Wir müssen uns zuerst fragen, wie die Ein- und Ausgabe des Netzwerkes kodiert sein soll. Wie alle neuronalen Netze besteht die Eingabe eines LSTM-Modells nicht aus Zeichen, sondern aus Gleitkommazahlen.

Die Eingabe soll ein Morse-Code sein. Da die Leerzeichen weggelassen wurden, besteht dieser nur aus zwei Zeichen. Ein häufiges Verfahren für die Kodierung kategorischer Daten ist das *one-hot encoding*. Die folgende Funktion implementiert diese Codierung der Eingabe, wobei die Leerzeichen im Morsecode automatisch weggelassen werden.

**Look-ahead** Es ist schwierig für das neuronale Netz, zu entscheiden, wo die Morse-Codes unterbrochen werden sollen, ohne wenigstens ein paar Zeichen vorwärts schauen zu können. Daher verschiebe ich einfach Ein- und Ausgabe mithilfe eines einstellbaren look-aheads gegeneinander. Anders gesagt, das neuronale Netz muss erst einige Zeichen später signalisieren, dass ein Leerzeichen eingefügt werden soll.

In [5]:
lookahead = 10

Die Eingabe wird als `float`-Array kodiert, damit man sie direkt ins neuronale Netzwerk einspeisen kann.

In [6]:
def encode_input(morse):
    encodings = { '.': [1, 0, 0], '-': [0, 1, 0], 'X': [0, 0, 1] }
    return np.array([encodings[x] for x in (morse #+ lookahead*"X" 
                                           )if x != ' ']).astype(float)

In [7]:
encode_input(to_morse("SOS"))

array([[1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.]])

Die Ausgabe des Netzwerkes möchte ich so machen, dass das Netz für jedes Eingabezeichen entscheiden soll, ob nach diesem Zeichen ein Leerzeichen wahrscheinlich ist. Die gewünschte Ausgabe ist also `1`, wenn auf ein Zeichen im ursprünglichen Morsecode ein Leerzeichen folgt, und sonst `0`. Für das letzte Zeichen macht es Sinn, `1` vorzuschreiben, da das Wortende ja auch ein Buchstabenende ist. Die folgende Funktion implementiert die gewünschte Ausgabe:

In [8]:
def compute_target(morse):
    return np.array(#lookahead*[0] + 
        [int((b == " ") | (b == "X")) for a,b in zip(morse[:-1], morse[1:]) if a != " "] + [1]).astype(float)

In [9]:
compute_target(to_morse("SOS V"))

array([0., 0., 1., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 1.])

Bei "SOS" haben alle Buchstaben drei Zeichen, deshalb besteht die gewünscht Ausgabe aus drei gleichen Teilen.

## Trainings-Daten generieren

In [10]:
with open("dataset0.txt", "r") as f:
    words = f.read()

In [11]:
assert all([x.upper() in morse_code or x == "\n" for x in words])

In [12]:
words = words.split("\n")[:-1]

In [13]:
def get_random_phrase(length):
    phrase = ""
    while len(phrase) < length:
        if phrase != "":
            phrase += " "
        phrase += words[np.random.randint(len(words))]
    return phrase
def make_training_data(length):
    phrase = get_random_phrase(length)
    morse = to_morse(phrase)
    inputs = encode_input(morse)
    targets = compute_target(morse)
    inputs = inputs[:length]
    targets = targets[:length]
    return inputs, targets

In [14]:
get_random_phrase(50)

'country inch performer gray kiss power occasion blog'

In [15]:
def make_batches(n_batches, length):
    inputs, targets = [np.zeros((n_batches, length, k)) for k in [3,1]]
    for i in range(n_batches):
        inputs[i], targets[i,:,0] = make_training_data(length)
    return inputs, targets

In [16]:
morse_all = [to_morse(w) for w in words]

In [17]:
np.amax([len(x) for x in morse_all])

50

In [18]:
words_all = np.zeros((len(words), 50 + lookahead, 3))
outs_all = np.zeros((len(words), 50 + lookahead, 1))

In [19]:
morse_all[1]

'.- --. . -. -.-. -.--'

In [20]:
for i in range(len(words)):
    c = morse_all[i]
    print(len(c), encode_input(c).shape)
    enc = encode_input(c)
    outs_all[i,lookahead:lookahead+enc.shape[0],0] = compute_target(c)
    words_all[i, :enc.shape[0]] = enc
    words_all[i, enc.shape[0]:] = [0,0,1]

27 (20, 3)
21 (16, 3)
25 (19, 3)
17 (13, 3)
23 (17, 3)
14 (10, 3)
8 (6, 3)
9 (7, 3)
13 (11, 3)
14 (11, 3)
15 (12, 3)
22 (16, 3)
10 (8, 3)
24 (18, 3)
25 (18, 3)
17 (13, 3)
24 (18, 3)
19 (15, 3)
20 (16, 3)
18 (13, 3)
19 (14, 3)
30 (22, 3)
14 (10, 3)
21 (16, 3)
20 (16, 3)
22 (17, 3)
24 (18, 3)
26 (19, 3)
28 (20, 3)
45 (33, 3)
28 (20, 3)
29 (22, 3)
35 (27, 3)
19 (14, 3)
28 (20, 3)
25 (19, 3)
38 (26, 3)
33 (24, 3)
21 (16, 3)
20 (15, 3)
14 (11, 3)
30 (22, 3)
11 (9, 3)
24 (18, 3)
24 (19, 3)
9 (7, 3)
25 (18, 3)
29 (21, 3)
32 (24, 3)
18 (13, 3)
19 (14, 3)
8 (6, 3)
7 (5, 3)
13 (10, 3)
16 (13, 3)
22 (18, 3)
34 (24, 3)
19 (14, 3)
33 (26, 3)
20 (14, 3)
16 (12, 3)
27 (21, 3)
30 (22, 3)
38 (28, 3)
30 (22, 3)
26 (19, 3)
40 (29, 3)
36 (25, 3)
11 (8, 3)
33 (24, 3)
17 (13, 3)
16 (12, 3)
24 (17, 3)
9 (7, 3)
19 (14, 3)
19 (14, 3)
29 (22, 3)
18 (13, 3)
14 (11, 3)
18 (12, 3)
7 (5, 3)
15 (10, 3)
22 (16, 3)
23 (15, 3)
32 (22, 3)
18 (13, 3)
21 (14, 3)
26 (20, 3)
20 (14, 3)
20 (15, 3)
30 (21, 3)
8 (6, 3)
23 (17,

In [21]:
words_all.shape, outs_all.shape

((1922, 60, 3), (1922, 60, 1))

## Trainings-Daten aus Text

In [22]:
with open("text.txt", "r") as f:
    text = f.read()
text = "".join([x for x in text if x == "\n" or x == " " or (x.upper() in morse_code)])
text = text.replace("\n", " ")
text = " ".join(text.split())

In [23]:
" ".join("abc def ghi".split())

'abc def ghi'

In [24]:
text[:500]

'The Project Gutenberg eBook of Bedouins by James Huneker This eBook is for the use of anyone anywhere in the United States and most other parts of the world at no cost and with almost no restrictions whatsoever You may copy it give it away or reuse it under the terms of the Project Gutenberg License included with this eBook or online at wwwgutenbergorg If you are not located in the United States you will have to check the laws of the country where you are located before using this eBook Title Be'

In [25]:
def make_batch_text(length):
    pos = np.random.randint(len(text) - length + 50)
    subtext = text[pos : pos + length + 50]
    idx = subtext[:50].find(" ")
    if idx > 0:
        subtext = subtext[idx : length + idx]
    morse = to_morse(subtext)
    inputs = encode_input(morse)
    targets = compute_target(morse)
    inputs = inputs[:length]
    targets = targets[:length]
    return inputs, targets

def make_batches_text(n_batches, length):
    inputs, targets = [np.zeros((n_batches, length, k)) for k in [3,1]]
    for i in range(n_batches):
        inputs[i], targets[i,:,0] = make_batch_text(length)
    return inputs, targets

## Morse-Dekodieren zum Testen

In [26]:
morse_inverse = { code: letter for letter, code in morse_code.items() }

In [27]:
def morse_decode_word(with_spaces):
    codes = with_spaces.split(" ")
    return "".join([morse_inverse[x] if x in morse_inverse else "?" for x in codes])
def morse_decode(s):
    return " ".join([morse_decode_word(x) for x in s.split("X")])

In [28]:
def remove_spaces(s):
    return "".join([x for x in s if x != " "])

In [29]:
def insert_spaces(s_no_spaces, output=None):
    inp_encode = encode_input(s_no_spaces + lookahead*"X")
    if output is None:
        output = model(inp_encode.reshape(1,-1,3)).numpy()[0,:,0][lookahead:]
    with_spaces = ""
    for i in range(len(s_no_spaces)):
        char = s_no_spaces[i]
        with_spaces += char
        if output[i] > 0 and i < len(s_no_spaces) - 1 and s_no_spaces[i+1] != "X" and s_no_spaces[i] != "X":
            with_spaces += " "
    return with_spaces

## Möglichkeiten generieren

In [30]:
def generate_possibilites(no_spaces):
    if no_spaces == "":
        return [""]
    poss = []
    for i in range(1, len(no_spaces)+1):
        if no_spaces[:i] in morse_inverse:
            lett = morse_inverse[no_spaces[:i]]
            poss_rest = generate_possibilites(no_spaces[i:])
            poss += [lett + x for x in poss_rest]
    return poss

In [31]:
generate_possibilites(remove_spaces(to_morse("just")))

['ETTTEETEEET',
 'ETTTEETEEA',
 'ETTTEETEIT',
 'ETTTEETEU',
 'ETTTEETIET',
 'ETTTEETIA',
 'ETTTEETST',
 'ETTTEETV',
 'ETTTEENEET',
 'ETTTEENEA',
 'ETTTEENIT',
 'ETTTEENU',
 'ETTTEEDET',
 'ETTTEEDA',
 'ETTTEEBT',
 'ETTTEAEEET',
 'ETTTEAEEA',
 'ETTTEAEIT',
 'ETTTEAEU',
 'ETTTEAIET',
 'ETTTEAIA',
 'ETTTEAST',
 'ETTTEAV',
 'ETTTEREET',
 'ETTTEREA',
 'ETTTERIT',
 'ETTTERU',
 'ETTTELET',
 'ETTTELA',
 'ETTTITEEET',
 'ETTTITEEA',
 'ETTTITEIT',
 'ETTTITEU',
 'ETTTITIET',
 'ETTTITIA',
 'ETTTITST',
 'ETTTITV',
 'ETTTINEET',
 'ETTTINEA',
 'ETTTINIT',
 'ETTTINU',
 'ETTTIDET',
 'ETTTIDA',
 'ETTTIBT',
 'ETTTUEEET',
 'ETTTUEEA',
 'ETTTUEIT',
 'ETTTUEU',
 'ETTTUIET',
 'ETTTUIA',
 'ETTTUST',
 'ETTTUV',
 'ETTTFEET',
 'ETTTFEA',
 'ETTTFIT',
 'ETTTFU',
 'ETTNETEEET',
 'ETTNETEEA',
 'ETTNETEIT',
 'ETTNETEU',
 'ETTNETIET',
 'ETTNETIA',
 'ETTNETST',
 'ETTNETV',
 'ETTNENEET',
 'ETTNENEA',
 'ETTNENIT',
 'ETTNENU',
 'ETTNEDET',
 'ETTNEDA',
 'ETTNEBT',
 'ETTNAEEET',
 'ETTNAEEA',
 'ETTNAEIT',
 'ETTNAEU',
 'ETTNAIE

In [32]:
[ to_morse(x) for x in words]

['.- -.-. - .. ...- .. - -.--',
 '.- --. . -. -.-. -.--',
 '.- -... .. .-.. .. - -.--',
 '.- -.-. - --- .-.',
 '.- -.-. - .-. . ... ...',
 '.- --. .-. . .',
 '.- .. --',
 '.- .. .-.',
 '.--- --- -...',
 '.- -... .-.. .',
 '.--. .-. . .--.',
 '.- .. .-. .-.. .. -. .',
 '.- -.. -..',
 '.- .. .-. .--. --- .-. -',
 '.- -.. -.. .. - .. --- -.',
 '.- .-.. .- .-. --',
 '.- -.. -.. .-. . ... ...',
 '.- .-.. -... ..- --',
 '.- .-- ..-. ..- .-..',
 '.- -.. -- .. .-. .',
 '.- -.-. -.-. . -. -',
 '.- -.. -- .. ... ... .. --- -.',
 '.- -.. -- .. -',
 '.- -.-. -.-. . .--. -',
 '.- .-.. .-.. --- .--',
 '.- -.-. -.-. . ... ...',
 '.- -.. ...- .- -. -.-. .',
 '.- -.-. -.-. .. -.. . -. -',
 '.- -.. ...- .- -. - .- --. .',
 '.- -.-. -.-. --- -- -- --- -.. .- - .. --- -.',
 '.- -.. ...- . -. - ..- .-. .',
 '.- .-.. .--. .... .- -... . -',
 '.- -.-. -.-. --- -- .--. .- -. -.--',
 '.- -.. ...- . .-. -',
 '.- -.. ...- . .-. - .. ... .',
 '.- -.-. -.-. --- ..- -. -',
 '.- -.. ...- . .-. - .. ... . -- . -. -',

In [33]:
len(np.unique(np.array([remove_spaces(to_morse(x)) for x in words], dtype=str)))

1854

In [34]:
len(words)

1922

## Definition und Training des Modells

In [35]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.LSTM(300, return_sequences=True))
#model.add(tf.keras.layers.Dense(150, activation='relu'))
#model.add(tf.keras.layers.LSTM(250, return_sequences=True))
#model.add(tf.keras.layers.Dense(500, activation='relu'))
model.add(tf.keras.layers.LSTM(400, return_sequences=True))
model.add(tf.keras.layers.Dense(1000, activation='relu'))
#model.add(tf.keras.layers.Dense(2000, activation='relu'))
#model.add(tf.keras.layers.LSTM(60, return_sequences=True))
model.add(tf.keras.layers.Dense(1))

2023-03-12 14:48:15.695577: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [36]:
model.compile(optimizer=tf.optimizers.Adam(), loss = tf.losses.BinaryCrossentropy(from_logits=True))

In [ ]:
n_iter = 1000
for i in range(n_iter):
    model.fit(words_all, outs_all, epochs = 1)
    for i in range(15,25):
        word = words[i]
        rec = morse_decode(insert_spaces(remove_spaces(to_morse(word))))
        print(word, rec)

61/61 [==============================] - 14s 191ms/step - loss: 0.2288
alarm ?T
address ?E
album ?T
awful ?E
admire ?E
accent ?T
admission ?E
admit ?
accept ?T
allow ?T
61/61 [==============================] - 11s 175ms/step - loss: 0.1610
alarm ?
address ?
album ?
awful ?
admire ?
accent ?
admission ?
admit ?
accept ?
allow ?
61/61 [==============================] - 11s 176ms/step - loss: 0.1542
alarm ?
address ?
album ?
awful ?
admire ?
accent ?
admission ?
admit ?
accept ?
allow ?
61/61 [==============================] - 11s 176ms/step - loss: 0.1771
alarm ?
address ?
album ?
awful ?
admire ?
accent ?
admission ?
admit ?
accept ?
allow ?
61/61 [==============================] - 11s 177ms/step - loss: 0.1665
alarm ?T
address P?E
album ??T
awful ??E
admire ?E
accent P?T
admission P?E
admit ?T
accept P?T
allow ??T
61/61 [==============================] - 12s 204ms/step - loss: 0.1646
alarm ?
address ?E
album ?T
awful ?E
admire ?E
accent ?T
admission ?
admit ?
accept ?T
allow ?
61/61 [=

61/61 [==============================] - 11s 172ms/step - loss: 0.1484
alarm ??
address P?H
album ??
awful ??L
admire P?L
accent P??
admission P?
admit PAX
accept P?
allow ??
61/61 [==============================] - 11s 173ms/step - loss: 0.1454
alarm AE?
address P?
album AE?
awful AE?
admire P?
accent P?
admission P?N
admit P?
accept P?
allow AE?
61/61 [==============================] - 11s 172ms/step - loss: 0.1690
alarm ?
address P?
album ?
awful R?
admire P?
accent ?
admission P?
admit P?
accept ?
allow ?
61/61 [==============================] - 11s 172ms/step - loss: 0.1529
alarm ?
address P?
album ?
awful R?
admire P?
accent ?
admission P?
admit P?
accept ?
allow ?
61/61 [==============================] - 11s 172ms/step - loss: 0.1515
alarm ??
address P?
album ??
awful ?
admire ??
accent P?
admission ??
admit P?
accept P?
allow ??
61/61 [==============================] - 10s 172ms/step - loss: 0.1474
alarm AEDETEY
address P?EEEESE
album AE?EEEJ
awful AE?EEETRE
admire PWEERE
accen

admit PADT
accept ?PT
allow ALREOW
61/61 [==============================] - 10s 171ms/step - loss: 0.1233
alarm ETLRY
address P?
album AAFSJ
awful ETPR?
admire P?L
accent ATRC?
admission P?ISON
admit PADT
accept W??
allow ETLREOW
61/61 [==============================] - 11s 172ms/step - loss: 0.1215
alarm ARFTW
address PREL?E
album AAERSJ
awful AWEREECE
admire PADRE
accent ACKERT
admission PADSISON
admit PADT
accept AKALANT
allow ETLLOAT
61/61 [==============================] - 12s 189ms/step - loss: 0.1250
alarm ALRTW
address P?E
album A?SJ
awful APR?E
admire P?RE
accent ACCRT
admission P?ISON
admit PADT
accept AT?PT
allow ALLOW
61/61 [==============================] - 11s 178ms/step - loss: 0.1191
alarm ARERTW
address PREL?E
album ARRSJ
awful APRE?E
admire PANERE
accent ACCRT
admission PA?IEION
admit PADT
accept A?LANT
allow ALLOW
61/61 [==============================] - 11s 187ms/step - loss: 0.1158
alarm ARFY
address PL?
album AR?J
awful APR?
admire PADL
accent ACC?
admission PA?IE

awful AWFUL
admire ADMIL
accent ACCENT
admission ADMISSION
admit ADMU
accept AC?ANT
allow ALLOW
61/61 [==============================] - 10s 166ms/step - loss: 0.0492
alarm ALA?
address A?LSS
album AR??
awful AWFUL
admire ADZL
accent ACCEK
admission ?MISSION
admit ADMU
accept A?PT
allow ALLOW
61/61 [==============================] - 10s 169ms/step - loss: 0.0510
alarm ALA?
address ADDRESS
album A?UM
awful AWFEARE
admire ADMIL
accent ACCRT
admission ?MISSION
admit ADMIT
accept ATR?EGT
allow ALLOW
61/61 [==============================] - 10s 167ms/step - loss: 0.0495
alarm ALA?
address ?DRESS
album AR?M
awful AWFURE
admire ADMIL
accent ACNNENT
admission WIMISSION
admit ADMIT
accept AK?ANT
allow ALLOW
61/61 [==============================] - 10s 167ms/step - loss: 0.0423
alarm ALA?
address ADDLSS
album A?ITM
awful AWFUL
admire ADMIL
accent ACCENT
admission WIMISSION
admit ADMIT
accept AKE?ANT
allow ALLOW
61/61 [==============================] - 10s 164ms/step - loss: 0.0391
alarm AAIARM
a

61/61 [==============================] - 10s 165ms/step - loss: 0.0103
alarm ALARM
address ?DRESS
album ALBUM
awful AWFUL
admire ADMIRE
accent ACCENT
admission ATIMISSION
admit ADMIT
accept WRCEPT
allow ALLOW
61/61 [==============================] - 10s 164ms/step - loss: 0.0105
alarm ALARM
address ?DRESS
album ALBUM
awful AWFUL
admire ADMIRE
accent ACCENT
admission ?MISSION
admit ADMIT
accept ACCEPT
allow ALLOW
61/61 [==============================] - 10s 166ms/step - loss: 0.0111
alarm ALARM
address ?DRESS
album ALBUM
awful AWFUL
admire ADMIRE
accent ACCENT
admission ?MISSION
admit ADMIT
accept ATRCEPT
allow ALLOW
61/61 [==============================] - 10s 165ms/step - loss: 0.0159
alarm ALAENM
address ?DRESS
album ALBITM
awful AWFUL
admire ADMIRE
accent ACCENT
admission ATIMISSION
admit ADMIT
accept ACCEWET
allow ETLLOW
61/61 [==============================] - 10s 166ms/step - loss: 0.0207
alarm ALARM
address ?DRESS
album ALBUM
awful AWFURE
admire ADMIRE
accent ACCENT
admission ?M

61/61 [==============================] - 10s 166ms/step - loss: 0.0080
alarm ALARM
address ?DRESS
album ALBUM
awful AWFUL
admire ADMIRE
accent ACCENT
admission ?ISSION
admit ADMIT
accept ACCEPT
allow ALLOW
61/61 [==============================] - 10s 167ms/step - loss: 0.0072
alarm ALARM
address ?DRESS
album ALBUM
awful AWFUL
admire ADMIRE
accent ACCENT
admission ?MISSION
admit ADMIT
accept WRCEPT
allow ALLOW
61/61 [==============================] - 10s 166ms/step - loss: 0.0065
alarm ALARM
address ?DRESS
album ALBUM
awful AWFUL
admire ADMIRE
accent ACCENT
admission ATIMISSION
admit ADMIT
accept ACCEPT
allow ALLOW
61/61 [==============================] - 10s 172ms/step - loss: 0.0062
alarm ALARM
address ?DRESS
album ALBUM
awful AWFUL
admire ADMIRE
accent ACCENT
admission WIMISSION
admit ADMIT
accept ACCEPT
allow ALLOW
61/61 [==============================] - 10s 171ms/step - loss: 0.0064
alarm ALARM
address ADDRESS
album ALBUM
awful AWFUL
admire ADMIRE
accent ACCENT
admission ATIMISSIO

61/61 [==============================] - 11s 181ms/step - loss: 0.0054
alarm ALARM
address ADDRESS
album ALBUM
awful AWFUL
admire ADMIRE
accent ACCENT
admission ATIMISSION
admit ADMIT
accept ATRCEPT
allow ALLOW
61/61 [==============================] - 11s 183ms/step - loss: 0.0054
alarm ALARM
address ADDRESS
album ALBUM
awful AWFUL
admire ADMIRE
accent ACCENT
admission ATIMISSION
admit ADMIT
accept ACCEPT
allow ALLOW
61/61 [==============================] - 11s 176ms/step - loss: 0.0054
alarm ALARM
address ?DRESS
album ALBUM
awful AWFUL
admire ADMIRE
accent ACCENT
admission ?MISSION
admit ADMIT
accept ACCEPT
allow ALLOW
61/61 [==============================] - 11s 178ms/step - loss: 0.0052
alarm ALARM
address ADDRESS
album ALBUM
awful AWFUL
admire ADMIRE
accent ACCENT
admission ADMISSION
admit ADMIT
accept ATRCEPT
allow ALLOW
61/61 [==============================] - 11s 181ms/step - loss: 0.0051
alarm ALARM
address ?DRESS
album ALBUM
awful AWFUL
admire ADMIRE
accent ACCENT
admission ?M

## Mehrere Möglichkeiten ausgeben

In [ ]:
for x in 1 / (1 + tf.exp(-model(encode_input(to_morse("airline")+lookahead*"X").reshape(1,-1,3)).numpy()[0,:,0])):
    print("{:.2f}".format(x))

In [ ]:
tf.nn.softmax([-10.0])

In [ ]:
def insert_spaces_options(s_no_spaces, n_options = 4):
    assert " " not in s_no_spaces
    inp_encode = encode_input(s_no_spaces + lookahead*"X")
    output = model(inp_encode.reshape(1,-1,3)).numpy()[0,:,0][lookahead:]
    print(inp_encode.shape, output.shape, len(s_no_spaces))
    options = []
    for j in range(n_options):
        with_spaces = ""
        for i in range(len(s_no_spaces)):
            char = s_no_spaces[i]
            with_spaces += char
            prob = 1 / (1 + np.exp(-output[i]))
            last_place = i == len(s_no_spaces) - 1
            if last_place: continue
            word_boundary = s_no_spaces[i] == "X" or s_no_spaces[i+1] == "X"
            if np.random.rand() < prob and not word_boundary:
                with_spaces += " "
        options.append(morse_decode(with_spaces))
    return options

In [ ]:
insert_spaces_options(remove_spaces(to_morse("addit")), 20)